In [1]:
import pandas as pd
import os
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import torch


/opt/anaconda3/envs/lora_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Test ASR implementation

In [26]:
## Load data
path = "/Users/reekaestacio/Documents/GitHub/Whisper_LLM_Bias/data"
df = pd.read_csv(path+"/stimuli.csv")
df["file_name"] = path + "/auditory_stimuli/" + df["id"].astype(str) + df["condition_id"] + ".mp3"

# Load pretrained model and processor
model_name = "facebook/wav2vec2-base-960h"  # or your fine-tuned model path
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def transcribe_mp3(file_path, target_sr=16000):
    # Load and resample audio
    waveform, sample_rate = torchaudio.load(file_path)
    if sample_rate != target_sr:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
        waveform = resampler(waveform)
    
    # Mono-channel and flatten
    input_values = processor(waveform.squeeze(), sampling_rate=target_sr, return_tensors="pt").input_values

    # Predict
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)

    # Decode to text
    transcription = processor.decode(predicted_ids[0])
    return transcription


In [33]:
# Run model
df["prediction"] = df["file_name"].apply(lambda x: transcribe_mp3(x))

In [34]:
# standardize labels
# w2v2 labels with uppercase
df['sentence'] = df['sentence'].str.upper()
df['sentence'] = df['sentence'].str.replace(".", "", regex=False)



num_incorrect = 0
accurate_list = []

# labels accurate/inaccurate
for _, row in df.iterrows():
    if row["sentence"] == row["prediction"]:
        accurate_list.append(1)
    else:
        accurate_list.append(0)
        num_incorrect += 1

df["accurate"] = accurate_list
print(f"{num_incorrect} incorrect predictions.")

# see inaccuracies
incorrect_df = df[df['accurate']==0]
incorrect_df

## spelling differences in wav2vec, not a problem for real dataset because of simple, single word utterances
## only 2 real mislabels


12 incorrect predictions.


,id,condition_id,condition_name,sentence,file_name,prediction,accurate
20,5,A,expected,THE CHEF SHARPENED THE KNIFE,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE CHEFF SHARPENED THE KNIFE,0
21,5,B,phonologically related,THE CHEF SHARPENED THE NIGHT,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE CHEFF SHARPENED THE NIGHT,0
22,5,C,semantically related,THE CHEF SHARPENED THE BLADE,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE CHEFF SHARPENED THE BLADE,0
23,5,D,both,THE CHEF SHARPENED THE KNIVES,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE CHEFF SHARPENED THE KNIVES,0
24,5,E,neither,THE CHEF SHARPENED THE SUN,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE CHEFF SHARPENED THE SUN,0
25,6,A,expected,THE LUMBERJACK CHOPPED THE TREE,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE LUMBER JACK CHOPPED THE TREE,0
26,6,B,phonologically related,THE LUMBERJACK CHOPPED THE TREAT,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE LUMBER JACK CHOPPED THE TREAT,0
27,6,C,semantically related,THE LUMBERJACK CHOPPED THE LOG,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE LUMBER JACK CHOPPED THE LOG,0
28,6,D,both,THE LUMBERJACK CHOPPED THE TRUNK,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE LUMBER JACK CHOPPED THE TRUNK,0
29,6,E,neither,THE LUMBERJACK CHOPPED THE COFFEE,/Users/reekaestacio/Documents/GitHub/Whisper_L...,THE LUMBER JACK CHOPPED THE COFFEE,0


In [137]:
# Try g2p mapping
from g2p_en import G2p

def get_phones(sen):
    '''
    convert texts to phone sequence
    '''
    sen = g2p(sen)
    sen = [re.sub(r'\d','',p) for p in sen]
    return sen

# Try LoRA

In [56]:
import os
import sys
# import soundfile
import random
import pandas as pd
import torch
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
from datasets import Dataset, Audio
from sklearn.model_selection import KFold
from peft import LoraConfig, get_peft_model


In [136]:
# Configure LoRA arguments
lora_config = LoraConfig(
    r=16, # can change rank, r=8 is most common
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"] # attention layers
)

# Add LoRA layers to Wav2Vec2 model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 589,824 || all params: 94,986,144 || trainable%: 0.6210


In [59]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

### Preprocess dataset

Not currently working, probably because test data is `.mp3` and not `.wav`. Try with lab data.

In [130]:
lora_df = df[['file_name', "sentence"]]
dataset = Dataset.from_pandas(lora_df)
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['file_name', 'sentence'],
        num_rows: 32
    })
    test: Dataset({
        features: ['file_name', 'sentence'],
        num_rows: 8
    })
})

In [131]:
def preprocess(example):
    # Load audio
    waveform, sr = torchaudio.load(example["file_name"])
    
    # Resample
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)

    # Flatten list of waveforms for correct dimensionality
    waveform = waveform.squeeze(0)  # shape becomes [samples]

    # Use the feature extractor (not full processor) — output is a dict with lists
    inputs = processor.feature_extractor(
        waveform,
        sampling_rate=16000,
        padding=False, # ensures files are the same size
        return_attention_mask=True, # tells the model what is actually audio and what is padding
        return_tensors=None  
    )

    # Flatten input_values from [[...]] → [...] for correct dimensionality
    input_values = inputs["input_values"][0]
    attention_mask = inputs["attention_mask"][0]

    # Tokenize word transcripts (label_ids)
    with processor.as_target_processor():
        label_ids = processor.tokenizer(
            example["sentence"].upper(), # upper to ensure correct tokenization
            return_tensors=None,
            padding=False,
            truncation=True
        ).input_ids

    return {
        # keep all inputs lists
        "input_values": input_values,
        "attention_mask": attention_mask,
        "labels": label_ids  
    }

In [ ]:
# NoneType error, possibly because mp3 and not wav file
processed_dataset = dataset['train'].map(preprocess, remove_columns=["file_name", "sentence"])

Map:   0%|          | 0/32 [00:00<?, ? examples/s]/opt/anaconda3/envs/lora_env/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map:   0%|          | 0/32 [00:00<?, ? examples/s]


ValueError: type of None unknown: <class 'NoneType'>. Should be one of a python, numpy, pytorch or tensorflow object.

### Train

In [60]:
## Load processor
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Load data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [138]:
import json

# Load your vocab.json
with open("vocab.json", "r") as f:
    vocab = json.load(f)

# Convert all keys to uppercase
vocab_upper = {k.upper(): v for k, v in vocab.items()}

# Save the updated vocab back to vocab.json (or a new file if you want to preserve the original)
with open("vocab_upper.json", "w") as f:
    json.dump(vocab_upper, f, indent=2)
